<a href="https://colab.research.google.com/github/Arka161/6390Kaggle1Project/blob/main/CalculatorLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Only blabbers generated result**

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

In [ ]:
with open('multiplication_table_dataset.csv', 'r', encoding='utf-8') as f:
    text = f.read()


In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [ ]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


0.20573 M parameters


In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

step 0: train loss 3.6651, val loss 3.6532
step 100: train loss 1.1040, val loss 1.3408
step 200: train loss 0.8723, val loss 1.1295
step 300: train loss 0.7297, val loss 0.9389
step 400: train loss 0.5580, val loss 0.8265
step 500: train loss 0.4731, val loss 0.6954
step 600: train loss 0.4206, val loss 0.6017
step 700: train loss 0.4070, val loss 0.6203
step 800: train loss 0.3863, val loss 0.5396
step 900: train loss 0.3725, val loss 0.5510
step 1000: train loss 0.3553, val loss 0.5538
step 1100: train loss 0.3504, val loss 0.5437
step 1200: train loss 0.3339, val loss 0.5367
step 1300: train loss 0.3251, val loss 0.4833
step 1400: train loss 0.3175, val loss 0.5286
step 1500: train loss 0.3105, val loss 0.5077
step 1600: train loss 0.3001, val loss 0.4937
step 1700: train loss 0.3002, val loss 0.5297
step 1800: train loss 0.2989, val loss 0.5375
step 1900: train loss 0.2852, val loss 0.4987
step 2000: train loss 0.2818, val loss 0.5050
step 2100: train loss 0.2801, val loss 0.5585


# **Trying out generating based on prompt**

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Set hyperparameters
batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

# Set random seed
torch.manual_seed(1337)

# Load and preprocess the data
with open('multiplication_table_dataset.csv', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out



class BigramLanguageModel(nn.Module):

  def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # ... (Rest of the code remains the same as provided)

  def generate_response(self, x, max_new_tokens):
        context = torch.tensor([x], dtype=torch.long, device=device)
        for _ in range(max_new_tokens):
            logits, _ = self(context)
            logits = logits[:, -1, :]  # becomes (B, C)
            probs = F.softmax(logits, dim=-1)  # (B, C)
            next_token = torch.multinomial(probs, num_samples=1)  # (B, 1)
            context = torch.cat((context, next_token), dim=1)  # (B, T+1)
        return context

# Create the model
model = BigramLanguageModel()
model = model.to(device)

# Create an optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# IGNORE THESE LINES
context = [stoi['W'], stoi['h'], stoi['a'], stoi['t'], stoi[' '], stoi['i'], stoi['s'], stoi[' '], stoi['4'], stoi['5'], stoi['x'], stoi['9'], stoi['?']]
generated_response = model.generate_response(context, max_new_tokens=50)
print("Generated Response:", decode(generated_response[0].tolist()))


step 0: train loss 3.6651, val loss 3.6532
step 100: train loss 1.1040, val loss 1.3408
step 200: train loss 0.8723, val loss 1.1295
step 300: train loss 0.7297, val loss 0.9389
step 400: train loss 0.5580, val loss 0.8265
step 500: train loss 0.4731, val loss 0.6954
step 600: train loss 0.4206, val loss 0.6017
step 700: train loss 0.4070, val loss 0.6203
step 800: train loss 0.3863, val loss 0.5396
step 900: train loss 0.3725, val loss 0.5510
step 1000: train loss 0.3553, val loss 0.5538
step 1100: train loss 0.3504, val loss 0.5437
step 1200: train loss 0.3339, val loss 0.5367
step 1300: train loss 0.3251, val loss 0.4833
step 1400: train loss 0.3175, val loss 0.5286
step 1500: train loss 0.3105, val loss 0.5077
step 1600: train loss 0.3001, val loss 0.4937
step 1700: train loss 0.3002, val loss 0.5297
step 1800: train loss 0.2989, val loss 0.5375
step 1900: train loss 0.2852, val loss 0.4987
step 2000: train loss 0.2818, val loss 0.5050
step 2100: train loss 0.2801, val loss 0.5585


IndexError: ignored

In [ ]:
# Double-check the chars variable
chars = sorted(list(set(text)))

# Create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}


# Create a function to encode the prompt into indices
def encode_prompt(prompt):
    prompt_tokens = prompt.split()
    prompt_indices = [stoi[token] if token in stoi else stoi[' '] for token in prompt_tokens]
    return prompt_indices


# Create a function to decode indices into text
def decode_indices(indices):
    tokens = [itos[idx] for idx in indices]
    return "".join(tokens)

# ... (rest of the code remains the same)

# Create a prompt for generating responses: 'What is 45x10?'
prompt = 'What is 45x10?'

# Encode the prompt into indices
prompt_indices = encode_prompt(prompt)

# Generate responses
if len(prompt_indices) > 0:
    generated_responses = model.generate_response(torch.tensor(prompt_indices, dtype=torch.long, device=device), max_new_tokens=20)
    for response_indices in generated_responses:
        generated_response = decode_indices(response_indices.tolist())
        print("Generated Response:", generated_response)
else:
    print("Prompt not found in vocabulary.")






TypeError: ignored

In [ ]:
chars

['\n',
 ' ',
 ',',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '=',
 '?',
 'I',
 'P',
 'R',
 'T',
 'W',
 'a',
 'e',
 'g',
 'h',
 'i',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'x']